In [1]:
import sys
import os
fname = r'C:\Users\agmontesb\Documents\GitHub\excel\tests\test_base_workbook.py'
sys.path.append(os.path.abspath(os.path.join(os.path.dirname(fname), '..')))
sys.path.append(os.path.abspath(os.path.join(os.path.dirname(fname), r'..\tests')))

In [2]:
import pytest
import openpyxl as px
import pandas as pd
import itertools
import re
import inspect
from typing import Any, Literal, Optional

In [3]:

from excel_workbook import (
    ExcelWorkbook, ExcelTable, 
    cell_address, cell_pattern, 
    tbl_address, rgn_pattern,
    XlErrors, TABLE_DATA_MAP, EMPTY_CELL, CIRCULAR_REF,
    tbl_pattern, rgn_pattern, flatten_sets
    )

from tests.utilities import TableComparator
import xlfunctions as xlf


In [4]:
from tests.fixtures import static_workbook as base_workbook

In [5]:
wb = inspect.unwrap(base_workbook)()
ws = wb.sheets[1]
tbl = ws.tables[1]
df = tbl.data

In [6]:
tbl

col,E,F,G,H
row,,,,
12,tabla 2:,,,
13,l1,25,100,0
14,l2,10,25,35
15,l3,15,38,53
16,l4,80,2438,2518
17,,105,2501,2606


In [7]:
tbl.get_formula('E12:H17')

col,E,F,G,H
row,,,,
12,tabla 2:,,,
13,l1,25,=+G6,=+$G$2*F13
14,l2,10,25,=+SUM(F14:G14)
15,l3,15,38,=+SUM(F15:G15)
16,l4,80,=+F9 + F13,=+SUM(F16:G16)
17,,=+F16+F15+F14,=+G16+G15+G14,=+H16+H15+H14


In [8]:
class XlValue:
    def __new__(cls, value):
        if isinstance(value, XlErrors):
            return value
        instanve = super().__new__(cls)
        instanve.value = value
        return instanve

    def __init__(self, value):
        self.value = value

    @staticmethod
    def xl_val_dec(func):
        def inner(self, other_in):
            other = other_in.value if isinstance(other_in, XlValue) else other_in
            cls = (type(1), type(1.0)) if isinstance(self.value, (type(1), type(1.0))) else type(self.value)
            if isinstance(other, cls):
                return func(self, other)
            return XlErrors.VALUE_ERROR
        return inner

    @xl_val_dec
    def __add__(self, other):
        return self.value + other

    @xl_val_dec
    def __radd__(self, other):
        return self.__add__(other)  # Delegate to __add__

    @xl_val_dec
    def __sub__(self, other):
        return self.value - other

    @xl_val_dec
    def __rsub__(self, other):
        return -self.__sub__(other)

    @xl_val_dec
    def __mul__(self, other):
        return self.value * other

    @xl_val_dec
    def __rmul__(self, other):
        return self.__mul__(other)

    @xl_val_dec
    def __truediv__(self, other):
        if other == 0:
            return XlErrors.DIV_ZERO_ERROR
        return self.value / other

    @xl_val_dec
    def __rtruediv__(self, other):
        if self.value == 0:
            return XlErrors.DIV_ZERO_ERROR
        return other / self.value

    @xl_val_dec
    def __eq__(self, other: object) -> bool:
        return isinstance(other, self.__class__) and self.value == other.value

    @xl_val_dec
    def __ne__(self, other: object) -> bool:
        return not self.__eq__(other)

    def __hash__(self):
        return super().__hash__()
    
    def __str__(self):
        return str(self.value)


In [9]:
val1 = XlValue(10)
val2 = XlValue(-10)
val3 =XlValue(XlErrors.DIV_ZERO_ERROR)

In [10]:
s1 = pd.Series([XlValue(x) for x in  [1, 2, 3, 4, 5]])
s2 = pd.Series([XlValue(x) for x in  [1, 'uno', 3, 0, 5]])

In [11]:
df = pd.DataFrame({'a': [1, 2, 3], 'b': [4, 5, 6]})
df

,a,b
0,1,4
1,2,5
2,3,6


In [12]:
def mySum(df, axis:Literal[0, 1] | None = None):
    if axis is None:
        return df.sum(axis=0).sum()
    elif axis == 0:
        return pd.Series([df.loc[:, x].sum() for x in df.columns], index=df.columns)
    else:
        return pd.Series([df.loc[x, :].sum() for x in df.index], index=df.index)

In [13]:
class strWrapper:
    fnc_names = [x for x in dir(pd.Series.str) if not x.startswith('_')]
    
    def __init__(self, s):
        self.s = s

    @staticmethod
    def str_dec(strfnc):
         def inner(*args, **kwargs):
            print(f'{args=}, {kwargs=}')
            answ = strfnc(*args, **kwargs)
            print(f'{answ=}')
            return answ.where(~answ.isna(), XlErrors.VALUE_ERROR)
         return inner

    def __getattr__(self, name):
           if name in self.fnc_names:
                return self.str_dec(getattr(self.s.str, name))
           raise AttributeError(f"'{name}' not found")

class MyClass:
    def __init__(self, s):
        self.s = s
    
    @property
    def str(self):
        s = pd.Series([x.value for x in self.s], index=self.s.index)
        return strWrapper(s)
    
    def __getattr__(self, name):
        return getattr(self.s, name)

In [14]:
S2 = MyClass(s2)
S2

In [15]:
from excel_workbook import pythonize_fml
range_ = [100000, 200000, 300000, 400000]
sum_range = [7000, 14000, 21000, 28000]
# sum_range = None
criteria = 300000
xlf.sumif(range_, criteria, sum_range)


np.int64(21000)

In [ ]:
criteria = str(criteria)
criteria = pythonize_fml(criteria, table_name='tbl')
if not criteria.startswith(('=', '>', '<')):
    criteria = f'=={criteria}'
criteria = f'range{criteria}'
criteria

In [30]:
sum_range = sum_range or range_
s = (
    pd.DataFrame({'range':range_, 'sum_range':sum_range})
    .query(criteria)
    .loc[:, 'sum_range']
    .sum()
)
s

np.int64(21000)

In [1]:
import re
import pandas as pd

In [2]:
texto = 'alex?mont~?es'
texto.count('?'), texto.count('~?'), texto.count('*'), texto.count('~*')

(2, 1, 0, 0)

In [14]:
import inspect

import xlfunctions as xlf

In [15]:
def xlFmlStr(tpl, with_kwargs=True):
    xlstr = lambda x: f"{chr(34)}{x}{chr(34)}" if x[::len(x)-1] != '""' else f"\'{x}\'"
    fnc = getattr(xlf, tpl[0])
    sig = inspect.Signature.from_callable(fnc)
    bsig = sig.bind(*tpl[1])
    if not with_kwargs or str(sig) == '(*data)':
        sig_str = ', '.join(
            f'{value}'
            for x in [*bsig.args, *bsig.kwargs.values()]
                            if (
                value := xlstr(x) if isinstance(x, str) else 
                '[' + ', '.join(
                            xlstr(y) if isinstance(y, str) else str(y).upper()
                            for y in x
                        ) + ']' if isinstance(x, list) else 
                str(x).upper()
            )
        )
    else:
        sig_str = ', '.join(
            f'{value}' if sig.parameters[key].default == inspect._empty else f'{key}={value}' 
            for key, x in bsig.arguments.items()
            if (
                value := xlstr(x) if isinstance(x, str) else 
                '[' + ', '.join(
                            xlstr(y) if isinstance(y, str) else str(y).upper()
                            for y in x
                        ) + ']' if isinstance(x, list) else 
                str(x).upper()
            )
        )
    return f"{tpl[0].replace('_', '').upper()}({sig_str})"


In [16]:
tpl = ('sumif', [["Vegetables", "Vegetables", "Fruits", "", "Vegetables", "Fruits"], '"Fruits"', [2_300, 5_500, 800, 400, 4_200, 1_200]], 2_000.0)
xlFmlStr(tpl, with_kwargs=False)

'SUMIF(["Vegetables", "Vegetables", "Fruits", "", "Vegetables", "Fruits"], \'"Fruits"\', [2300, 5500, 800, 400, 4200, 1200])'

In [25]:
criteria = '+234.25a'
criteria.translate(str.maketrans({'.': '', '-': '', '+':''})).isdigit()

False

In [ ]:
fmls = """=+$G$2*F13
=+F14+G14
=+SUM(F15:G15)
=+SUM(F16:G16)
=+H16+H15+H14""".split('\n')
fmls

In [ ]:
cells = '''$H$13
$H$14
$H$15
$H$16
$H$17
'''.replace('$', '').splitlines()
cells

In [ ]:
cell_to_delete = 'F'
changes = [f'{cell_to_delete}{i}' for i in range(13, 18)]
changes

In [ ]:
err_ref_cell = "'Hoja de Trabajo'!Z0"

In [ ]:
def pass_anchors(coord1, coord2):
    return coord2


In [ ]:
def shrink_range(linf, lsup, changes, del_what):
    bflag = del_what == 'col'
    k1, k2, fnc, gnc = (2, 3, ord, chr) if bflag else (3, 2, int, str)
    ucol = cell_pattern.match(lsup).group(k1)
    lx_row = cell_pattern.match(changes[0]).group(k2)
    row_changes = [
        tpl[k1 - 1] for x in changes 
        if (tpl := cell_pattern.match(x).groups()) and tpl[k2 - 1] == lx_row and tpl[k1 - 1] <= ucol
    ]
    if not row_changes:
        return linf, lsup
    min_col = fnc(row_changes[0]) if linf in changes else (fnc(row_changes[0]) - 1)
    n = len(row_changes)
    answ = [linf]
    for x in (linf, lsup)[1 - int(linf in changes):]:
        tpl = list(cell_pattern.match(x).groups())
        tpl[2 - int(bflag)] = gnc(max(min_col, (fnc(tpl[2 - int(bflag)]) - n)))
        sht, col, row = tpl
        x = f"'{sht}'!{col}{row}" if sht else f"{col}{row}"
        answ.append(x)

    return answ[-2:]


In [ ]:
del_action = 'col'
pattern = rgn_pattern
def replacement(m):
    if ':' in m[0]:
        m = tbl_pattern.match(m[0])
        prefix = f"'{m[1]}'!" if m[1] else ''
        linf, lsup = map(lambda x: f"{prefix}{x}", m[0].replace('$', '').split(':'))
        if err_ref_cell and all(x in changes for x in (linf, lsup)):
            return err_ref_cell
        linf, lsup = shrink_range(linf, lsup, changes, del_action)
        lsup = lsup.split('!')[-1]
        linf, lsup = map(lambda tpl: pass_anchors(*tpl), zip(m[0].split(':'), (linf, lsup)))
        sub_str = ':'.join([linf, lsup])
        return sub_str
    key = m[0].replace('$', '')
    return (err_ref_cell or pass_anchors(m[0], changes[key])) if key in changes else m[0]


In [ ]:
scase = ['ABC', 'EFG', 'CDEF'][0]
changes = [f'{col}{row}' for col in scase for row in range(13, 18)]
changes

In [ ]:
shrink_range('F15', 'G15', [f'G{x}' for x in range(13, 18)], 'col')

In [ ]:
def test_del_cases(scase, ncase, del_action):
        if del_action == 'col':
                changes = [f'{col}{row}' for col in scase for row in range(13, 18)]
        else:
                changes = [f'{chr(col)}{row}' for col in range(ord('A'), ord('H') + 1) for row in scase]
        fml = fmls[ncase]
        linf, lsup = rgn_pattern.search(fml)[0].split(':')
        l_answ, r_answ = shrink_range(linf, lsup, changes, del_action)
        return fml, (linf, lsup), (l_answ, r_answ)

In [ ]:
scase, ncase, del_action = range(15, 20), 0, 'row'
if del_action == 'col':
    answ = locals().get(f'del_{scase}').splitlines()[ncase]
    left, right = rgn_pattern.search(answ)[0].split(':')
else:
    fmls = '''=+SUM(A13:A17)	=+SUM(B13:B17)	=+SUM(C13:C17)	=+SUM(D13:D17)	=+SUM(E13:E17)	=+SUM(F13:F17)	=+SUM(G13:G17)'''.split('\t')
fml, tpl1, tpl2 = test_del_cases(scase, ncase, del_action)
print(fml, tpl1)
# print(tpl2, (left, right))
print(tpl2, ('left', 'right'))

In [ ]:
del_action = 'col'

for scase in ['ABC', 'EFG', 'CDEF']:
    for ncase in range(5):
        answ = locals().get(f'del_{scase}').splitlines()[ncase]
        left, right = rgn_pattern.search(answ)[0].split(':')
        fml, (linf, lsup), (l_answ, r_answ) = test_del_cases(scase, ncase, del_action)

        if not all(x == y for x, y in zip((left, right), (l_answ, r_answ))):
            print(f'*** {scase=}, {ncase=}')
            print(f'    {fml=}, {linf=}, {lsup=}, {left=}, {right=}')
            print(f'    ({l_answ=}, {l_answ == left}), ({r_answ=}, {r_answ == right})')

In [ ]:
fmls = base.split('\n')
fmls

In [ ]:
df = pd.DataFrame(fmls, columns=['fml'], index=pd.Index(cells, name='cell'))
df

In [ ]:

df.fml.str.replace(
    pattern,
    replacement,
    regex=True
)            


In [ ]:
import collections
list(collections.Counter(['B', 'C', 'B', 'C', 'A']).keys())

In [ ]:
base = '''=+SUM(A13:G13)
=+SUM(C14:G14)
=+SUM(A15:E15)
=+SUM(A16:D16)
=+SUM(A13:G17)'''


In [ ]:
del_ABC = '''=+SUM(A13:D13)
=+SUM(A14:D14)
=+SUM(A15:B15)
=+SUM(A16:A16)
=+SUM(A13:D17)'''

In [ ]:
del_EFG = '''=+SUM(A13:D13)
=+SUM(C14:D14)
=+SUM(A15:D15)
=+SUM(A16:D16)
=+SUM(A13:D17)'''

In [ ]:
del_CDEF = '''=+SUM(A13:C13)
=+SUM(C14:C14)
=+SUM(A15:B15)
=+SUM(A16:B16)
=+SUM(A13:C17)'''

In [ ]:
def xlf_code(arange):
    s = pd.Series(arange.flatten())
    return s.str[0].map(lambda x: ord(x))

In [ ]:
pd.Series(tbl['E12:E14'].values.flatten()).str[0].map(lambda x: ord(x))


In [ ]:
from xlfunctions import *

In [ ]:
def xlf_lower(*arange):
    s = pd.Series(pd.Series(arange).values.flatten())
    return s.str.lower()


In [ ]:
pd.Series([tbl['E12'].values]).values

In [ ]:
xlf_lower(tbl['E12'].values)

In [ ]:
tbl